#Fusión de Bases de Datos del Panóptico de Twitch v0.1.0

La siguiente aplicación tiene como objetivo el recopilar las bases de datos generadas durante la primera fase experimental y construir una única base de datos interpretable utilizando cualquier aplicación de análisis estadístico.

Si bien el diseño de esta aplicación impide su uso sobre bases de datos no estructurados bajo el formato propuesto en el Panóptico de Twitch, es posible realizar modificaciones a la estructura del texto para poder aplicar el modelo a cualquier formato de base de datos basada en CSV.

Panóptico de Twitch v0.1.0: https://drive.google.com/file/d/1XiJ9mDe6EcDp5lYHilbDIABkYlHJPD-A/view?usp=sharing

In [ ]:
import socket
import time
import re
import glob
import pandas as pd
import os.path
from os import path

from google.colab import drive
drive.mount('/content/drive')

output_dir = "/content/drive/MyDrive/Panoptico_Twitch/"
df_concat = pd.DataFrame()
exc = 0
all_filename_count = 0
extension = 'csv'
output_dir_var = ""
gender = 0

**Selección de Streamers**

Selecciona el nombre de los streamers a fusionar en la base de datos. Genera la lista "streamers" que se utilizará en el futuro para distintas funciones (selección de carpeta, renombrar, generizar, etc).



In [ ]:
streamers = "auronplay", "iamcristinini", "ibai", "illojuan", "rivers_gg", "staryuuki"

**Primera fusión: Bases de Datos de cada Streamer**

Este algoritmo se compone de una estructura de tres bucles:
  1. Selección de streamer
  2. Selección de día
  3. Carga en la memoria y concatenación de los datos en .CSV

En definitiva, la aplicación generará un archivo "Fusión_[Nombre].csv" por cada streamer explicitado en la lista "streamers".

In [ ]:
for streamrs in streamers:
  os.chdir(output_dir + streamrs)

  list_days = os.listdir()
  print(list_days)

  for j in list_days:
    j = str(j)
    print(output_dir, streamrs,"/",j,sep='')
    output_dir_var = output_dir + streamrs + "/" + j

    os.chdir(output_dir_var)

    all_filenames = [z for z in glob.glob('*.{}'.format(extension))]
    
    for i in range(0,len(all_filenames)):
      if os.path.isfile(all_filenames[i]) and os.path.getsize(all_filenames[i]) > 1:
        df_concat = pd.concat([pd.read_csv(all_filenames[i], low_memory=False, on_bad_lines='skip'), df_concat], ignore_index=True)
        exc = exc + 1
        all_filename_count = all_filename_count + len(all_filenames)

    print(df_concat)
    print(exc, "/", len(all_filenames))
    all_filename_count = 0
    exc = 0

  output_dir = "/content/drive/MyDrive/Panoptico_Twitch/"
  os.chdir(output_dir)
  df_concat.to_csv("Fusion_" + streamrs + ".csv", index=False, encoding='utf-8-sig')
  df_concat = pd.DataFrame()

**Generización y Nombre**

Debido a que en la tabulación de los datos en el Panóptico de Twitch no se establecen datos referidos al streamer, ésta función persigue establecer datos relevantes indicados en la investigación en la tabulación: para permitir la comparación entre streamers, en primer lugar, adjuntamos el nombre del canal dodne se escribió cada mensaje; y el género de streamer, codificado en éste algoritmo



In [ ]:
output_dir = "/content/drive/MyDrive/Panoptico_Twitch/"
os.chdir(output_dir)

for streamrs in streamers:

  if streamrs in ("illojuan", "ibai", "auronplay"):
    gender = "male" 
  if streamrs in ("iamcristinini", "rivers_gg", "staryuuki"):
    gender = "female"

  file = "Fusion_" + streamrs + ".csv"

  df = pd.DataFrame()

  df = pd.read_csv(file, low_memory=False)

  df["streamer"] = streamrs
  df["gender"] = gender

  print(df)

  df.to_csv("Fusion_" + streamrs + "_final.csv", index=False, encoding='utf-8-sig')


**Segunda fusión: Generación de la Base de Datos final**

Esta fusión se compone, sencillamente, de la carga de las bases de datos previamente generadas y la creación de un nuevo archivo "Base_de_datos.csv".

In [ ]:
output_dir = "/content/drive/MyDrive/Panoptico_Twitch/"
os.chdir(output_dir)
df = pd.DataFrame()

for streamrs in streamers:
  file = "Fusion_" + streamrs + "_final.csv"

  df = pd.concat([pd.read_csv(file, low_memory=False, on_bad_lines='skip'), df], ignore_index=True)
  print(df)

df.to_csv("Base_de_datos.csv", index=False, encoding='utf-8-sig')


**Limpieza de la Base de Datos**



In [ ]:
with open('Base_de_datos.csv', 'r') as in_file, open('Base_de_datos_limpia.csv', 'w') as out_file:
    seen = set() # set for fast O(1) amortized lookup
    for line in in_file:
        if line in seen: continue # skip duplicate

        seen.add(line)
        out_file.write(line)

Función de limpieza de Base de Datos.

In [ ]:
all_filenames = [z for z in glob.glob('*.{}'.format(extension))]

for i in all_filenames:
  if i != "Base_de_datos_limpia.csv":
    print("Eliminando ", i, "...")
    os.remove(i)